In [1]:
# ----------------------
# Load required packages
# ----------------------
import copy
import gc
import sys, getopt
import json as json
import random
import emcee
import datetime
import corner
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp
from multiprocessing import Pool
from covid19model.models import models
from covid19model.optimization.run_optimization import checkplots, calculate_R0
from covid19model.optimization.objective_fcns import prior_custom, prior_uniform
from covid19model.data import mobility, sciensano, model_parameters
from covid19model.optimization import pso, objective_fcns
from covid19model.models.time_dependant_parameter_fncs import ramp_fun
from covid19model.visualization.output import _apply_tick_locator 
from covid19model.visualization.optimization import autocorrelation_plot, traceplot
from covid19model.models.time_dependant_parameter_fncs import  make_vaccination_function, vacc_strategy, mobility_update_func
from covid19model.models.utils import read_pops, read_coordinates_nis, initial_state


# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [ ]:
import sys
sys.path

# Data

In [2]:
# ---------
# Load data
# ---------

# Contact matrices
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(dataset='willem_2012')
Nc_all = {'total': Nc_total, 'home':Nc_home, 'work': Nc_work, 'schools': Nc_schools, 'transport': Nc_transport, 'leisure': Nc_leisure, 'others': Nc_others}
levels = initN.size
# Sciensano data
df_sciensano = sciensano.get_sciensano_COVID19_data(update=False)
# Google Mobility data
df_google = mobility.get_google_mobility_data(update=False)
# Model initial condition on September 1st
warmup = 0
with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/initial_states_2020-09-01.json', 'r') as fp:
    initial_states = json.load(fp)  
# Samples dictionary
with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/BE_WAVE2_BETA_COMPLIANCE_2021-03-02.json', 'r') as fp:
    samples_dict = json.load(fp)

# Time-dep fun

In [3]:
# --------------------------------------
# Time-dependant social contact function
# --------------------------------------

# Extract build contact matrix function
from covid19model.models.time_dependant_parameter_fncs import make_contact_matrix_function, ramp_fun
contact_matrix_4prev, all_contact, all_contact_no_schools = make_contact_matrix_function(df_google, Nc_all)

# Define policy function
def policies_wave2(t, states, param, l , tau, prev_schools, prev_work, prev_rest, prev_home, scenario=0, contact_increase=1):
    
    # Convert tau and l to dates
    tau_days = pd.Timedelta(tau, unit='D')
    l_days = pd.Timedelta(l, unit='D')

    # Define key dates of first wave
    t1 = pd.Timestamp('2020-03-15') # start of lockdown
    t2 = pd.Timestamp('2020-05-15') # gradual re-opening of schools (assume 50% of nominal scenario)
    t3 = pd.Timestamp('2020-07-01') # start of summer holidays
    t4 = pd.Timestamp('2020-09-01') # end of summer holidays

    # Define key dates of second wave
    t5 = pd.Timestamp('2020-10-19') # lockdown (1)
    t6 = pd.Timestamp('2020-11-02') # lockdown (2)
    t7 = pd.Timestamp('2020-11-16') # schools re-open
    t8 = pd.Timestamp('2020-12-18') # Christmas holiday starts
    t9 = pd.Timestamp('2021-01-04') # Christmas holiday ends
    t10 = pd.Timestamp('2021-02-15') # Spring break starts
    t11 = pd.Timestamp('2021-02-21') # Spring break ends
    t12 = pd.Timestamp('2021-02-28') # Contact increase in children
    if scenario == 0:
        # Paasvakantie as is
        t13 = pd.Timestamp('2021-04-02') 
        t14 = pd.Timestamp('2021-04-18') 
    elif scenario == 1:
        # Extend one week before
        t13 = pd.Timestamp('2021-03-26') 
        t14 = pd.Timestamp('2021-04-18')
    elif scenario == 2:
        # Extend one week after
        t13 = pd.Timestamp('2021-04-02') 
        t14 = pd.Timestamp('2021-04-25')
    elif scenario == 3:
        # Shift one week
        t13 = pd.Timestamp('2021-03-26') 
        t14 = pd.Timestamp('2021-04-11')
    elif scenario == 4:
        # No easter holiday
        t13 = pd.Timestamp('2021-04-02') 
        t14 = pd.Timestamp('2021-04-02') 
    elif scenario == 5:
        # Close schools
        t13 = pd.Timestamp('2021-04-02') 
        t14 = pd.Timestamp('2021-06-30') 
        
    t15 = pd.Timestamp('2021-07-01') # Summer holiday starts

    t = pd.Timestamp(t.date())
    # First wave
    if t <= t1:
        return all_contact(t)
    elif t1 < t < t1 + tau_days:
        return all_contact(t)
    elif t1 + tau_days < t <= t1 + tau_days + l_days:
        policy_old = all_contact(t)
        policy_new = contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                                    school=0)
        return ramp_fun(policy_old, policy_new, t, tau_days, l, t1)
    elif t1 + tau_days + l_days < t <= t2:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=0)
    elif t2 < t <= t3:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=0)
    elif t3 < t <= t4:
        return contact_matrix_4prev(t, school=0)
    # Second wave
    elif t4 < t <= t5 + tau_days:
        return contact_matrix_4prev(t, school=1)
    elif t5 + tau_days < t <= t5 + tau_days + l_days:
        policy_old = contact_matrix_4prev(t, school=1)
        policy_new = contact_matrix_4prev(t, prev_schools, prev_work, prev_rest, 
                                    school=1)
        return ramp_fun(policy_old, policy_new, t, tau_days, l, t5)
    elif t5 + tau_days + l_days < t <= t6:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=1)
    elif t6 < t <= t7:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=0)
    elif t7 < t <= t8:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=1) 
    elif t8 < t <= t9:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=0)
    elif t9 < t <= t10:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=1)
    elif t10 < t <= t11:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=0)    
    elif t11 < t <= t12:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=1)
    elif t12 < t <= t13:
        mat = contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, school=1)
        mat = np.array(mat)
        mat[:,0] = (contact_increase+1)*mat[:,0]
        mat[:,1] = (contact_increase+1)*mat[:,1]
        return mat
    elif t13 < t <= t14:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=0) 
    elif t14 < t <= t15:
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=1) 
    else:
        t = pd.Timestamp(t.date())
        return contact_matrix_4prev(t, prev_home, prev_schools, prev_work, prev_rest, 
                              school=0)

# ------------------------------
# Function to add binomial draws
# ------------------------------

def add_poisson(state_name, output, n_samples, n_draws_per_sample, UL=1-0.05*0.5, LL=0.05*0.5):
    data = output[state_name].sum(dim="Nc").values
    # Initialize vectors
    vector = np.zeros((data.shape[1],n_draws_per_sample*n_samples))
    # Loop over dimension draws
    for n in range(data.shape[0]):
        binomial_draw = np.random.poisson( np.expand_dims(data[n,:],axis=1),size = (data.shape[1],n_draws_per_sample))
        vector[:,n*n_draws_per_sample:(n+1)*n_draws_per_sample] = binomial_draw
    # Compute mean and median
    mean = np.mean(vector,axis=1)
    median = np.median(vector,axis=1)    
    # Compute quantiles
    LL = np.quantile(vector, q = LL, axis = 1)
    UL = np.quantile(vector, q = UL, axis = 1)
    return mean, median, LL, UL

# -----------------------------------------
# Define sampling function with vaccination
# -----------------------------------------

def draw_fcn_vacc(param_dict,samples_dict):
    # Calibrated samples
    idx, param_dict['beta'] = random.choice(list(enumerate(samples_dict['beta'])))
    param_dict['da'] = samples_dict['da'][idx]
    param_dict['omega'] = samples_dict['omega'][idx]
    param_dict['sigma'] = 5.2 - samples_dict['omega'][idx]
    param_dict['tau'] = samples_dict['tau'][idx] 
    param_dict['l'] = samples_dict['l'][idx] 
    param_dict['prev_schools'] = samples_dict['prev_schools'][idx]   
    param_dict['prev_home'] = samples_dict['prev_home'][idx]      
    param_dict['prev_work'] = samples_dict['prev_work'][idx]       
    param_dict['prev_rest'] = samples_dict['prev_rest'][idx]
    # Vaccination parameters
    param_dict['e_i'] = np.random.uniform(low=0,high=1) # Vaccinated individual can either be non-infectious or equally infectious as a non-vaccinated individual 
    param_dict['e_s'] = np.random.uniform(low=0.85,high=0.95) # Vaccine results in a 85-95% lower susceptibility
    param_dict['e_h'] = np.random.uniform(low=0.5,high=1.0) # Vaccine blocks hospital admission between 50-100%
    param_dict['refusal'] = np.random.triangular(0.05, 0.20, 0.40, size=9) # min. refusal = 5%, max. refusal = 40%, expectation = 20%
    # Variant parameters
    param_dict['K_inf'] = np.random.uniform(low=1.25,high=1.40)
    return param_dict

# Spatial model

In [4]:

# Spatial stratification
agg = 'arr'

# Interaction matrices
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = \
    model_parameters.get_interaction_matrices(spatial=agg)

# Load the parameters using `get_COVID19_SEIRD_parameters()`.
params_vacc = model_parameters.get_COVID19_SEIRD_parameters(vaccination=True, spatial=agg)

params_vacc.update({'l': 21, 'tau': 21, 'prev_schools': 0, 'prev_work': 0.5, 'prev_rest': 0.5,
               'prev_home': 0.5, 'zeta': 8*30})

start_date = pd.to_datetime('2020-03-18')


# Add parameters for the daily update of mobility
params_vacc.update({'agg' : agg,
               'default_mobility' : None}) # mobility defaults to average mobility

# Add clearly distinct infectivity parameters for demonstration
beta_R = 0.0
beta_U = 0.0
beta_M = 0.035
params_vacc.update({'beta_R' : beta_R,
               'beta_U' : beta_U,
               'beta_M' : beta_M})

In [42]:
# Initial states: single 30 year old exposed individual in Brussels
initE = initial_state(dist=81000, agg=agg, age=40, number=1000) # 1000 40-somethings dropped in Arlon
initial_states = {'S': initN, 'E': initE}

sciensano_first_dose, df_sciensano_start, df_sciensano_end = make_vaccination_function(df_sciensano)


# Add the time-dependant parameter function arguments
vacc_order = np.array(range(9))
vacc_order = vacc_order[::-1]
daily_dose = 30000
refusal = 0.3*np.ones(9)
delay = 21
d_vacc = 12*30 # duration of vaccine protection

# Update with additional parameters for social policy function
params_vacc.update({'l': 21, 'tau': 21, 'prev_schools': 0, 'prev_work': 0.5, 'prev_rest': 0.5,
               'prev_home': 0.5, 'zeta': 8*30, 'contact_increase': 0})
# Update with additional parameters for vaccination
params_vacc.update(
    {'scenario': 0, 'vacc_order': vacc_order, 'daily_dose': daily_dose,
     'refusal': refusal, 'delay': delay, 'df_sciensano_start': df_sciensano_start,
     'df_sciensano_end': df_sciensano_end, 'sciensano_first_dose': sciensano_first_dose}
)

# Initiate model with initial states, defined parameters, and lockdown_func determining the evolution of Nc
model_vacc = models.COVID19_SEIRD_spatial_vacc(initial_states, params_vacc, time_dependent_parameters = \
                                           {'Nc' : policies_wave2, 'place' : mobility_update_func, 'N_vacc': vacc_strategy}, spatial=agg)


In [43]:
# --------------------------
# Initialize the new variant
# --------------------------
start_sim = '2020-09-01'
K_inf = 1.30
K_hosp = 1.00
Re_1feb = 0.958*K_inf
incubation_period = 5.2
n_periods = 14/incubation_period

model_vacc.parameters.update({'K_inf': K_inf,
                              'K_hosp': K_hosp,
                              'injection_day': (pd.Timestamp('2021-01-14') - pd.Timestamp(start_sim))/pd.Timedelta('1D'),
                              'injection_ratio': (K_inf-1)/(Re_1feb**n_periods)})

In [ ]:
end_sim = '2021-09-01'
start_sim = pd.to_datetime('2020-01-01')
out = model_vacc.sim(end_sim, start_date=start_sim)